In [1]:
from baseband_analysis.analysis.toa import get_TOA
import numpy as np
import os
import glob
from chime_frb_api import frb_master
import datetime as dt
from baseband_analysis.core import BBData
import gc
from baseband_analysis.core.signal import tiedbeam_baseband_to_power
from baseband_analysis.core.dedispersion import coherent_dedisp
from baseband_analysis.analysis.snr import get_snr
from astropy.time import Time
from IPython.display import clear_output

--------------------------------------------------------------------------
[[19564,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: 1bae6bb3b670

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------


In [2]:
basepath = '/data/user-data/ksand/R3/fitburst_run_final/'
entries = os.listdir(basepath)
entries.remove('.ipynb_checkpoints')
entries = np.array(entries).astype(int)
entries = np.sort(entries)
print(entries)

[ 23789294  23891929  40363753  40750224  40750656  60073678  65395717
  65511496  67522832  67522901  69509443  71665813  71784400  71784833
  71785167  73726599  83984528  84081108  92773515  92844954 135577833
 151677988 153859596 154910364 154913824 155079679 163181014 163362088
 163534005 166812476 171681248 171855709 175004577 175929272 176060009
 180666196 180667327 188553320 194488374 194488393 194488454 195132396
 200973875 203798724 203935225]


In [3]:
basepath = '/data/user-data/ksand/R3/fitburst_run_final/'
event_ids = []
pm_files = []
result_files = []
input_files = []
summary = []
for event in entries:
    if (event != '.ipynb_checkpoints'): 
        path_to_event = os.path.join(basepath, str(event))
        trials = np.array(os.listdir(path_to_event))
#         print(event, len(trials))
        count = 0


        for trial in trials:
            if (trial != '.ipynb_checkpoints'):
                path_to_file = os.path.join(path_to_event, trial)
                #print(event, path_to_file, "\n")
                #print(path_to_file)
                event_ids.append(int(event))
                np.array(pm_files.append(glob.glob(path_to_file + '/pm*.npz')))
                #print(pm_files)
                np.array(result_files.append(glob.glob(path_to_file + '/result*.npz')))
                np.array(input_files.append(glob.glob(path_to_file + '/R3*.npz')))
                np.array(summary.append(glob.glob(path_to_file + '/summary*.png')))
                count = count + 1

In [4]:
DMs = []
downsamps = []
for pm_file in pm_files:
    if len(pm_file) > 0:
        data_pm = np.load(pm_file[0], allow_pickle = True)
        dm = data_pm['arr_0'].item(0)['DM']
        ds = data_pm['arr_0'].item(0)['ds']
        DMs.append(float(dm))
        downsamps.append(int(ds))
print(DMs)
DMs = np.array(DMs)
print(np.argmax(DMs))
downsamps = np.array(downsamps)

[348.732, 348.791, 348.92, 349.89, 348.813, 348.68, 349.11, 349.89, 349.847, 349.097, 348.98, 349.27, 349.37, 349.725, 348.84, 348.73, 349.1944387797214, 349.46, 348.955, 348.792, 350.12, 348.99, 348.8435, 348.838, 348.77503143294547, 349.47, 349.542, 348.83, 348.98, 349.6, 348.93, 348.95, 348.97, 349.38, 348.8165, 348.89570588235296, 349.062, 349.79, 349.9, 349.0277461483363, 348.862, 348.94, 349.2, 349.05, 348.901]
20


In [ ]:
TOAs_bb_fixDM = []
for i in range(len(entries)):
    clear_output(wait=True)
    frb_master_base_url = "http://frb-vsop.chime:8001"
    master = frb_master.FRBMaster(base_url = frb_master_base_url)

    event_id = entries[i]
    print(event_id)
    r = master.events.get_event(event_number=int(event_id))

    burst_time_str = r['measured_parameters'][0]['datetime'][0:19]
    burst_time = dt.datetime.strptime(burst_time_str, "%Y-%m-%d %H:%M:%S")
    print(burst_time)

    year = burst_time.strftime("%Y")
    month = burst_time.strftime("%m")
    day = burst_time.strftime("%d")

    file = '/data/chime/baseband/processed/{}/{}/{}/astro_{}/singlebeam_{}.h5'.format(year, month, day, event_id, event_id)
    gc.collect()
    print(file)

    print("Loading singelebeam file {}".format(file))
    data = BBData.from_file(file)

    if "DM" in data["tiedbeam_baseband"].attrs.keys():
        DM0 = data["tiedbeam_baseband"].attrs["DM"]
        print("Tiedbeam baseband dm is", DM0)
    else:
        DM0 = 0

    try:
        if DM0 > 1:
            print("Data is already de-smeared")
            DM_coh  = data['tiedbeam_power'].attrs['DM_coherent']
            DM_coh = 382.82
            print("performing coherent dedispersion")
            tiedbeam_baseband_to_power(data,time_downsample_factor=1,dm = DM_coh,dedisperse=True)           
            coherent_dedisp(data, DM_coh)
        else:
            print("Data is not de-smeared, will not perform coherent-dedispersion")

    except KeyError:
        print('Coherent de-dispersion was not done on this event.')
        if DM is not None:
            print('Performing oherent de-dispersion...')
            tiedbeam_baseband_to_power(data,time_downsample_factor=1,dm = DM,dedisperse=True)           
            coherent_dedisp(data, DM)
            try:
                data['tiedbeam_power'].attrs['DM_coherent']
            except KeyError:
                print('Coherent de-dispersion failed.')
            else:
                print('Coherent de-dispersion performed successfully.')
        else:
            raise ValueError('Please supply the DM to use for coherent de-dispersion!')

    # freq_id, freq, power, _, _, valid_channels, _, DM, downsampling_factor = get_snr(data, 
    #                                                                                  DM = DMs[i],
    #                                                                                  downsample=downsamps[i],
    #                                                                                  fill_missing_time = None,
    #                                                                                  diagnostic_plots=False,
    #                                                                                  spectrum_lim = False,
    #                                                                                  return_full=True,
    #                                                                                  DM_range = None,
    #                                                                                  lte_mask = True,
    #                                                                                  raise_missing_signal = False,
    #                                                                                  do_rfi_clean = True
    #                                                                                 )

    TOA = get_TOA(data, DM=348.82, downsampling_factor=downsamps[i])#, power=power, first_valid_bin=0)
    t = Time(TOA, format='unix')
    TOAs_bb_fixDM.append(t.mjd)
    print(t.mjd)

[2023-01-15 02:25:10,501] INFO Authorization Status: None
[2023-01-15 02:25:10,502] INFO Authorization Method: Tokens


23789294


[2023-01-15 02:25:10,820] INFO Authorization Token : Expired
[2023-01-15 02:25:10,821] INFO Reauthorize Method: Tokens
[2023-01-15 02:25:11,311] WARNING It is required that you pass in a value for the "algorithms" argument when calling decode().
[2023-01-15 02:25:11,312] INFO Reauthorize Result: Passed


2018-12-25 03:53:03
/data/chime/baseband/processed/2018/12/25/astro_23789294/singlebeam_23789294.h5
Loading singelebeam file /data/chime/baseband/processed/2018/12/25/astro_23789294/singlebeam_23789294.h5


In [50]:
TOA_mjd_bb = []
for t in TOAs_bb:
    x = Time(t, format='unix')
    TOA_mjd_bb.append(x.mjd)
print(TOA_mjd_bb)
TOA_mjd_bb = np.array(TOA_mjd_bb)
oname = 'R3_BB_TOAs'
np.savez(oname, TOA_mjd_bb)

[58477.16185106008, 58478.15520959571, 58638.716433091045, 58639.70561269861, 58639.71008089679, 58786.32074905812, 58835.17323655196, 58836.17197680902, 58852.1362833927, 58852.137731946015, 58868.0758577926, 58882.04680524802, 58883.039950144615, 58883.04405354814, 58883.0537194434, 58899.007059797106, 58981.77662303259, 58982.771575046674, 59013.69287125802, 59014.68532475482, 59111.406430915566, 59225.104278723964, 59241.060714461775, 59244.04399640264, 59244.062294394615, 59245.05832696872, 59275.96877324059, 59276.962570132644, 59277.96033745101, 59306.89010034776, 59355.747585411315, 59357.74680372812, 59390.652026868105, 59406.600247855546, 59407.59936336849, 59440.50725934581, 59440.515848079536, 59486.389633362036, 59519.30410440554, 59519.30469754965, 59519.30908634315, 59521.28870800225, 59552.2118273624, 59569.162656417844, 59570.15497822374]


In [53]:
print(TOAs_bb_fixDM)
oname = 'R3_BB_TOAs_fixDM'
np.savez(oname, TOAs_bb_fixDM)

[58477.161851118966, 58478.15520959953, 58638.716433091045, 58639.70561256462, 58639.71008118883, 58786.32074908104, 58835.17323644542, 58836.17197676984, 58852.13628319428, 58852.13773192104, 58868.07585775069, 58882.046805156475, 58883.03995011363, 58883.04405336504, 58883.05371943958, 58899.00705983135, 58981.77662293164, 58982.771575030754, 59013.69287125226, 59014.68532475957, 59111.40643055071, 59225.10427870111, 59241.06071445797, 59244.04399639504, 59244.06229440225, 59245.058326938, 59275.968773202236, 59276.962570109885, 59277.96033744144, 59306.89010020224, 59355.74758540366, 59357.74680372805, 59390.65202685285, 59406.60024778695, 59407.59936336876, 59440.50725933439, 59440.51584807371, 59486.38963320218, 59519.30410438974, 59519.30469752334, 59519.30908634479, 59521.288707977525, 59552.21182733184, 59569.162656413915, 59570.15497821943]


In [13]:
import numpy as np

data1 = np.load('R3_BB_TOAs.npz')
print(data1['arr_0'])
data2 = np.load('R3_BB_TOAs_fixDM.npz')
print(data2['arr_0'])
print(data1['arr_0'][20],data2['arr_0'][20],data1['arr_0'][20]-data2['arr_0'][20])

diff = np.abs(data1['arr_0'] - data2['arr_0'])
print(np.max(diff))

[58477.16185106 58478.1552096  58638.71643309 58639.7056127
 58639.7100809  58786.32074906 58835.17323655 58836.17197681
 58852.13628339 58852.13773195 58868.07585779 58882.04680525
 58883.03995014 58883.04405355 58883.05371944 58899.0070598
 58981.77662303 58982.77157505 59013.69287126 59014.68532475
 59111.40643092 59225.10427872 59241.06071446 59244.0439964
 59244.06229439 59245.05832697 59275.96877324 59276.96257013
 59277.96033745 59306.89010035 59355.74758541 59357.74680373
 59390.65202687 59406.60024786 59407.59936337 59440.50725935
 59440.51584808 59486.38963336 59519.30410441 59519.30469755
 59519.30908634 59521.288708   59552.21182736 59569.16265642
 59570.15497822]
[58477.16185112 58478.1552096  58638.71643309 58639.70561256
 58639.71008119 58786.32074908 58835.17323645 58836.17197677
 58852.13628319 58852.13773192 58868.07585775 58882.04680516
 58883.03995011 58883.04405337 58883.05371944 58899.00705983
 58981.77662293 58982.77157503 59013.69287125 59014.68532476
 59111.406